In [ ]:
import pandas as pd
from pathlib import Path
from IPython.display import clear_output
import re
from collections import Counter
import nltk
from nltk.corpus import stopwords
from natasha import Segmenter, NewsEmbedding, NewsMorphTagger, NewsSyntaxParser, NewsNERTagger, MorphVocab, Doc

# Вариант загрузки нескольких файлов из под-директории /temp относительно текущей

In [ ]:
# конвеер загрузки нескольких xls файлов
column = 'Текст'
sheet_name = 'Base'

df = pd.DataFrame()
for final_xlsx in sorted(list(Path('./temp').rglob('*.xlsx'))):
    df_temp = pd.read_excel(final_xlsx, sheet_name = sheet_name, index_col=0)[[column]].astype({column: 'string'})
    df = pd.concat([df, df_temp])
df.reset_index(drop=True, inplace=True)
df

# Вариант загрузки одного файла

In [ ]:
# загрузка одного файла
# название файла с текстовыми данными
file_request = 'текстовые_данные_для_частотного_анализа.xlsx'
# имя листа с данными в Excel-файле
sheet_name = 'Лист1'
# имя столбца с текстовыми данными
column = 'Текст'

df = pd.read_excel(file_request, sheet_name = sheet_name)[[column]].astype({column: 'string'})
df.head()

---

# Основной блок

In [ ]:
nltk.download('stopwords')
stopWords = set(stopwords.words('russian'))
glagolitsa = 'АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ'
allowed = set(' ' + '_' + '-' + glagolitsa + glagolitsa.lower())

In [ ]:
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

In [ ]:
def lemmatize_normalize(text: str) -> str:
    
    # Stage_1. Выделение именованных сущностей
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    doc.tag_ner(ner_tagger)
    for span in doc.spans:
        span.normalize(morph_vocab)
    normalize_name = ['_'.join(word.split()) if len(word.split()) > 1 else word for word in [_.normal for _ in doc.spans]]
    unnormalize_name = [word.text for word in doc.spans]
    assert len(normalize_name)==len(unnormalize_name), 'Assert Error'
    
    # Stage_2.  Удаление именованных сущностей из исходного корпуса
    my_list=[]
    for i, elem in enumerate(unnormalize_name):
        my_list.append((elem,'_' + str(i)))
        if elem in text:
            text = text.replace (elem, '_' + str(i) + ' ')
            
    idx = [elem[1] for elem in my_list]  # эмпирические номера именованных объектов
            
    # Stage_3.  Лемматизация корпуса после удаления именованных сущностей
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    doc.tag_ner(ner_tagger)
    
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
        
    text = [_.lemma for _ in doc.tokens]
    
    # Stage_4. Сборка
    final_list=[normalize_name[int(word[1:])] if word in idx else word for word in text]
     
    return ' '.join(final_list)

In [ ]:
def clean_text(text: str) -> str:

    # Stage_1. Only russian text, ' ', '_' and '-'
    text = ' '.join(''.join(l for l in text if l in allowed).split())

    # Stage_2. Delete stopWords and short_words
    text = ' '.join([word for word in text.split() if word not in stopWords and len(word) > 2])

    return text

In [ ]:
df['Lemma_corpus'] = df.apply(lambda x: clean_text(lemmatize_normalize(str(x[column]))), axis=1)
corpus = df['Lemma_corpus'].str.cat(sep=' ')
unique_key = set(corpus.split())
unique_values = [0] * len(unique_key)
unique_dict = dict(zip(unique_key, unique_values))
print(len(corpus),' слов, из них ',len(unique_key),' уникальных. П = ', len(corpus)*len(unique_key))

In [ ]:
count = 0
lentgh = len(unique_key)*df.shape[0]
for word in unique_key:
    for row in df['Lemma_corpus'].iteritems():
        count+=1
        if count % 1000000 == 0:
            clear_output()
            print(count,'done. Всего -',lentgh,'. ',round((count/lentgh)*100,2),' %')
        if word in row[1].split():
            unique_dict[word] += 1
            
data = pd.DataFrame([unique_dict]).T.sort_values(0, ascending=False)
with pd.ExcelWriter('freq_analiz_sp.xlsx', options={'strings_to_urls': True}) as writer:  
    data.to_excel(writer, sheet_name='Лист1')